In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from airbnb_priceforecaster.data import AirBnBDataset

# Exploring the AirBnB listings

The aim is to get an understanding of how each feature is distributed - taking note of the following:

- Missing values
- Categorical or continuous?
- Skewed distributions

## Steps

1. Download data
2. Decide what to keep and what to drop


# Downloading the data

1. Download from [download url](http://data.insideairbnb.com/denmark/hovedstaden/copenhagen/2020-05-30/data/listings.csv.gz)
2. Unzip and place into the `data/raw` folder
3. Rename it to `listings_2020_05_30.csv`

# Drop columns

We have 100+ features - many of which are obviously not relevant to our task, such as the host id. There are many features that might be included in a much more advanced model or for a different usecase. For our usecase, we can only include data that one of our potential customers would have - so including factors such as reviews or host picture is probably not useful at this point. Any text descriptions or host-provided prose is not included for now, though could potentially be used in another model to serve as input for this one.

We can always come back and add more features as our model evolves - so it's important to write down what we know about each feature and update it when we know more

1. Decide what columns to keep
2. Document what each feature is and first impressions

In [3]:
raw_data = pd.read_csv("../data/raw/listings_2020_05_30.csv")

/home/anders/.pyenv/versions/3.8.3/envs/priceforecaster/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (43,61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
raw_data.sample(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
23738,35366161,https://www.airbnb.com/rooms/35366161,20200530152033,2020-06-03,"Bright, spacy and clean, close to center CPH","Bright, spacy and clean, Copenhagen apartment ...",NaN,"Bright, spacy and clean, Copenhagen apartment ...",none,Amager is a neighborhood in rapid development....,...,t,f,moderate,f,f,1,1,0,0,0.85
5515,7452457,https://www.airbnb.com/rooms/7452457,20200530152033,2020-06-01,Celebrate Christmas in lovely CPH!,Cosy and bright apartment with everything you ...,The apartment will be all yours the time you s...,Cosy and bright apartment with everything you ...,none,The apartment is located perfectly for a stay ...,...,f,f,flexible,f,f,1,1,0,0,0.05
20120,27391608,https://www.airbnb.com/rooms/27391608,20200530152033,2020-06-02,Unique penthouse apartment in Copenhagen,"This unique, modern and newly renovated apartm...",NaN,"This unique, modern and newly renovated apartm...",none,NaN,...,f,f,flexible,f,f,1,1,0,0,0.54
13682,19149086,https://www.airbnb.com/rooms/19149086,20200530152033,2020-05-31,Amazing penthouse in the heart of vibrant Nørr...,Luxurious penthouse in the heart of vibrant Nø...,NaN,Luxurious penthouse in the heart of vibrant Nø...,none,The city centre is 7 minutes away by bus 5C or...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.59
1158,1859074,https://www.airbnb.com/rooms/1859074,20200530152033,2020-06-01,"Nice studio with balcony, free parking and roo...","Cosy 1 room apartment perfect for couples, sol...",NaN,"Cosy 1 room apartment perfect for couples, sol...",none,NaN,...,f,f,moderate,f,f,1,1,0,0,0.74
3142,4737250,https://www.airbnb.com/rooms/4737250,20200530152033,2020-06-02,Amazing flat in the heart of Vesterbro,NaN,Lovely flat with lots of soul.,Lovely flat with lots of soul. Located right i...,none,The central train station is less than ten min...,...,t,f,flexible,f,f,2,2,0,0,0.09
26233,39515193,https://www.airbnb.com/rooms/39515193,20200530152033,2020-06-03,Lys og rummelig lejlighed,Boligen er beliggende på Teglholmen med kort a...,Boligen indeholder 3 værelser og stor stue og ...,Boligen er beliggende på Teglholmen med kort a...,none,Teglholmen/Sluseholmen ligger langs Københavns...,...,t,f,moderate,f,f,1,1,0,0,0.37
17186,23231855,https://www.airbnb.com/rooms/23231855,20200530152033,2020-05-31,"Big Bright Room in Frederiksberg, Fasanvej Sta...",AREA Stay in Frederiksberg one of the coolest ...,THE ROOM The room is aproxx. 30 sqm. It contai...,AREA Stay in Frederiksberg one of the coolest ...,none,NaN,...,t,f,moderate,f,f,2,0,2,0,1.06
4357,6523022,https://www.airbnb.com/rooms/6523022,20200530152033,2020-06-01,Charming and modern flat,The flat is located in the lovely green and ch...,The flat is located in the lovely green and ch...,The flat is located in the lovely green and ch...,none,Frederiksberg has a reputation of being very s...,...,t,f,moderate,f,f,1,1,0,0,0.46
23000,34100919,https://www.airbnb.com/rooms/34100919,20200530152033,2020-06-02,Charming house close to everything!,Very charming house in two stories with a litt...,NaN,Very charming house in two stories with a litt...,none,NaN,...,t,f,strict_14_with_grace_period,f,f,70,70,0,0,0.75


I went through each column, writing down what I could see from a cursory glance [here](../docs/features/features.rst)

The final list of features to include for now is:

In [5]:
usecols = [
    "house_rules",
    "host_since",
    "host_location",
    "host_response_time",
    "host_neighbourhood",
    "host_listings_count",
    "host_total_listings_count",
    "host_verifications",
    "host_has_profile_pic",
    "host_identity_verified",
    "neighbourhood",
    "zipcode",
    "latitude",
    "longitude",
    "is_location_exact",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "bed_type",
    "amenities",
    "square_feet",
    "price",
    "security_deposit",
    "cleaning_fee",
    "guests_included",
    "extra_people",
    "minimum_nights",
    "maximum_nights",
    "instant_bookable",
    "cancellation_policy",
    "require_guest_profile_picture",
    "require_guest_phone_verification",
]

# Starting work on dtypes

It's good practice to map the correct dtypes from the start, this will save processing time over the data, and can prevent memory explosions.

Note that we are not setting their final form - some need to be loaded as strings, for example, so that we can handle missing values properly and then convert to their final form. 

This is documented [here](../docs/features/keep_features.rst)

In [6]:
raw_df = pd.read_csv("../data/raw/listings_2020_05_30.csv", usecols=usecols)

/home/anders/.pyenv/versions/3.8.3/envs/priceforecaster/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
raw_df.head()

,house_rules,host_since,host_location,host_response_time,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,...,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,No smoking allowed! No pets.,2009-05-12,"Copenhagen, Capital Region of Denmark, Denmark",NaN,Nørrebro,1.0,1.0,"['email', 'phone', 'reviews']",t,f,...,$0.00,$33.00,1,$67.00,2,15,f,moderate,f,f
1,We will leave the house clean and in good and ...,2010-04-17,"Copenhagen, Capital Region of Denmark, Denmark",NaN,Indre By,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,...,"$5,000.00","$1,100.00",3,$350.00,3,30,f,moderate,f,f
2,Please be respectful to the neighbors and keep...,2010-04-22,"Copenhagen, Capital Region of Denmark, Denmark",within an hour,Indre By,4.0,4.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,...,"$3,727.00",$522.00,1,$0.00,3,31,f,moderate,f,f
3,Smoking is allowed on the balcony only. Pleas...,2010-05-15,"Copenhagen, Capital Region of Denmark, Denmark",within a day,Vesterbro,1.0,1.0,"['email', 'phone', 'reviews']",t,f,...,NaN,$300.00,1,$0.00,7,14,f,strict_14_with_grace_period,f,f
4,Please respect that this is our home.,2010-05-18,"Copenhagen, Capital Region of Denmark, Denmark",NaN,Østerbro,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,...,NaN,$75.00,1,$0.00,7,31,t,moderate,f,f


In [8]:
mapping = {
    "house_rules": "string",
    "host_since": "string",
    "host_location": "string",
    "host_response_time": "category",
    "host_acceptance_rate": "string",
    "host_neighbourhood": "category",
    "host_listings_count": "Int64",
    "host_total_listings_count": "Int64",
    "host_has_profile_pic": "string",
    "host_identity_verified": "string",
    "neighbourhood": "category",
    "zipcode": "string",
    "latitude": "float64",
    "longitude": "float64",
    "property_type": "category",
    "accommodates": "Int8",
    "bathrooms": "float32",
    "bedrooms": "Int8",
    "beds": "Int8",
    "bed_type": "category",
    "amenities": "string",
    "square_feet": "Int64",
    "price": "string",    
    "security_deposit": "string",
    "cleaning_fee": "string",
    "guests_included": "Int8",
    "extra_people": "string",
    "minimum_nights": "Int64",
    "maximum_nights": "Int64",
    "instant_bookable": "string",
    "cancellation_policy": "category",
    "require_guest_profile_picture": "string",
    "require_guest_phone_verification": "string"
}

In [9]:
raw_df = pd.read_csv("../data/raw/listings_2020_05_30.csv", usecols=mapping.keys(), dtype=mapping)

In [10]:
processed_df = raw_df.copy()

In [11]:
raw_df.head().iloc[:, 15:]

,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,2,1.0,1,1,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",97,$368.00,$0.00,$33.00,1,$67.00,2,15,f,moderate,f,f
1,6,1.5,4,4,Real Bed,"{TV,Wifi,Kitchen,""Indoor fireplace"",Heating,""F...",<NA>,"$2,398.00","$5,000.00","$1,100.00",3,$350.00,3,30,f,moderate,f,f
2,12,2.5,6,7,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",<NA>,"$3,095.00","$3,727.00",$522.00,1,$0.00,3,31,f,moderate,f,f
3,2,1.0,1,1,Real Bed,"{Wifi,Kitchen,""Paid parking off premises"",Heat...",<NA>,$797.00,<NA>,$300.00,1,$0.00,7,14,f,strict_14_with_grace_period,f,f
4,4,1.0,3,3,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",<NA>,$855.00,<NA>,$75.00,1,$0.00,7,31,t,moderate,f,f


# Preprocessing columns

Now the data is loaded optimally, we can start looking at necessary data cleanup. At this stage, we are not doing feature engineering, merely fixing flaws and converting to final form of data types.

In [12]:
def convert_price(series):
    return pd.to_numeric(series.str.replace("$", "").str.replace(",", ""), errors="coerce").astype("float64")

def convert_percent(series):
    return pd.to_numeric(series.str.replace("%", ""), errors="coerce").astype("Int8")

def convert_bool(series):
    return series.map({"t": True, "f": False}).astype("bool")

## host_acceptance_rate
We need to remove the `%` sign from each column and convert to an integer

In [13]:
processed_df["host_acceptance_rate"] = convert_percent(processed_df["host_acceptance_rate"])

# host_has_profile_pic

We need to convert `f` to False and `t` to True

In [14]:
processed_df["host_has_profile_pic"] = convert_bool(processed_df.host_has_profile_pic)

# host_identity_verified

We need to convert `f` to False and `t` to True

In [15]:
processed_df["host_identity_verified"] = convert_bool(processed_df["host_identity_verified"])

## price

Remove the `,` and `$` sign and convert to correct dtype

In [16]:
processed_df["price"] = convert_price(processed_df["price"])

## security_deposit

Remove the `,` and `$` sign and convert to correct dtype

In [17]:
processed_df["security_deposit"] = convert_price(processed_df["security_deposit"])

## cleaning_fee

Remove the `,` and `$` sign and convert to correct dtype

In [18]:
processed_df["cleaning_fee"] = convert_price(processed_df["cleaning_fee"])

## extra_people
Remove the `,` and `$` sign and convert to correct dtype

In [19]:
processed_df["extra_people"] = convert_price(processed_df["extra_people"])

## instant_bookable

In [20]:
processed_df["instant_bookable"] = convert_bool(processed_df["instant_bookable"])

## require_guest_profile_picture

In [21]:
processed_df["require_guest_profile_picture"] = convert_bool(processed_df["require_guest_profile_picture"])

## require_guest_phone_verification


In [22]:
processed_df["require_guest_phone_verification"] = convert_bool(processed_df["require_guest_phone_verification"])

# Extracting Features

Now that the main dataset is ready, it's time to extract the necessary features from the data

## Amenities

We need to extract all possible categories and create a column for each, which we can then one-hot encode.
So first, we must extract all possible categories

In [23]:
def one_hot_encode_amenities(df: pd.DataFrame):
    return (
        df.amenities
            .str.slice(start=1, stop=-1)  # Remove the "{}" at the front and back
            .str.replace('"', '')  # Remove quoting of multi-word amenities
            .str.split(",")
            .apply(lambda x: pd.Series(index=set(x), data=1))  # For each row, make a Series of
            # 1s. `apply` will concatenate
            # the result
            .fillna(0)
            .astype("bool")
            .rename(columns=lambda x: "_".join(x.split(" ")).lower())  # Rename all columns to
        # lowercase snake_case
    )

In [24]:
def convert_amenities(df: pd.DataFrame):
    one_hot_df = one_hot_encode_amenities(df)
    return df.join(one_hot_df).drop(columns="amenities")

In [25]:
processed_df = convert_amenities(processed_df)

## House Rules

A hypothesis is that more rules impact the price. We can start by counting the lenght of the description to represent having more rules

In [27]:
processed_df["house_rules_len"] = processed_df.house_rules.str.len()

In [34]:
import httpx

In [30]:
import asyncio

In [40]:
url = r"https://dawa.aws.dk/postnumre/reverse"

In [51]:
lats_longs = processed_df.loc[processed_df.zipcode.isna(), ["latitude", "longitude"]].to_dict(orient="records")

In [70]:
async def get_zipcode(series, client):
    await asyncio.sleep(0.1)
    resp = await client.get(url, params={"x": series["longitude"], "y": series["latitude"]})
    if resp.status_code == 200:
        return resp.json()["nr"]
    if resp.is_error:
        raise ValueError(resp.text)

In [73]:
async def get_all_zipcodes(lats_longs):
    async with httpx.AsyncClient() as client:
        tasks = []
        for lat_lon in lats_longs:
            tasks.append(get_zipcode(lat_lon, client))
        return await asyncio.gather(*tasks)

In [72]:
result = await get_all_zipcodes(lats_longs)

808


## Zipcode

There are some missing zipcodes - we can look them up using DAWA and the reverse geocoding API

In [ ]:
processed_df.loc[processed_df.zipcode.isna(), ["latitude", "longitude"]]